In [16]:
from synsense.net.auto import AutoSNN
from synsense.net.dataset import Dataset_Texture_Stream
import torch.nn as nn

Split train and test data into lists

In [2]:
from user import split_train_test
root = r'J:\coop\data\20150115_stream_us\processed'
train_size = 0.7
random_seed = 42
train_list, test_list = split_train_test(root, train_size, random_seed)

Dataset creation

In [15]:
device = "pc"
gridsize = (20, 20)
after_crop_size = (260, 260, 1)
n_time_bins = 40

train_dataset = Dataset_Texture_Stream(train_list, device, gridsize, after_crop_size, n_time_bins)
test_dataset = Dataset_Texture_Stream(test_list, device, gridsize, after_crop_size, n_time_bins)

print(len(train_dataset), len(test_dataset))
print(train_dataset[0][0].shape, train_dataset[0][1])

700 300
torch.Size([40, 1, 20, 20]) tensor(1)


In [ ]:
batch_size = 60
auto = AutoSNN(batch_size)
model_path = r'./models/event_stream_us.pt'
device = "cuda:0"
k_folds = 2
epochs = 7
cnn = nn.Sequential(
    nn.Conv2d(1, 8, 3, 1, bias=False),
    nn.ReLU(),
    nn.AvgPool2d(2, 2),
    nn.Conv2d(8, 16, 3, 1, bias=False),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(64, 128, bias=False),
    nn.ReLU(),
    nn.Linear(128, 10, bias=False)
)

train_accs, val_accs = auto.train(model_path, cnn, train_dataset, device, k_folds, epochs)